# Yapay Sinir Ağları

Bu yazıyı ve kodları hazırlarken, Michael Nielson'ın açık kaynak Neural Networks and Deep Learning adlı [kitabından](http://neuralnetworksanddeeplearning.com) yararlandım.


### Numpy hatırlayalım

In [1]:
import random
import numpy as np

![](NumPy.png)

In [2]:
e = np.array([[1,1],
              [2,2],
              [3,3,]])
f = np.array([[1],
              [0]])

print(np.shape(e))
print(np.shape(f))
print(e.dot(f))

(3, 2)
(2, 1)
[[1]
 [2]
 [3]]


In [3]:
print(e,"\n")
print(e.T,"\n")

[[1 1]
 [2 2]
 [3 3]] 

[[1 2 3]
 [1 2 3]] 



In [4]:
for a, b in zip(e, e.T):
    print(a,b)

[1 1] [1 2 3]
[2 2] [1 2 3]


In [5]:
dVeri = np.array([1,2,3])
def dFonk():
    dVeri = 2 * dVeri
print(dVeri)

[1 2 3]


In [6]:
dVeri = 2 * dVeri
print(dVeri)

[2 4 6]


In [7]:
A = [1,3,5,7]
B = ["a","b","x","y"]
for a,b in zip(A,B):
    print(a,b)

1 a
3 b
5 x
7 y


#### Yardimci Fonksiyonlar

Sıgmoid fonksiyonu bir nöronun gelen toplam girdiyi, 0 ile 1 arasında bie çıktı değerine dönüştürür. Bu fonksiyonun türevi, kendisi ile 1den cıkarılmış halinin çarpınıma eşittir.

In [8]:
#### Yardimci Fonksiyonlar
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))
def sigmoid_turevi(z):
    return sigmoid(z)*(1-sigmoid(z))

### Ağı oluşturalım
Her bir katmanda kaç sinir hücresi olacağını bir listeden alarak, ağı oluşturan bir fonksiyon yazalım.

In [9]:
def ag(katmanlar):
    b = [np.random.randn(k, 1) for k in katmanlar[1:]] # bias degerleri (ilk katman haric)
    W = [np.random.randn(k2, k1) for k1, k2 in zip(katmanlar[:-1],katmanlar[1:])]
    return W, b

#### Örnek bir ağ

In [10]:
katmanlar = [3, 4, 2]
agirlik, bias = ag(katmanlar)

print("agirlik :")
for w in agirlik:
    print(w, "\n")
    
print("bias :")
for b in bias:
    print(b, "\n")

agirlik :
[[-0.81677266 -1.26841745  1.70132266]
 [-0.91724354 -0.71945923 -0.12453133]
 [ 0.84350872  0.83501493 -0.03834903]
 [ 0.85558418 -0.08470351  0.17014328]] 

[[ 0.43575331 -1.30116637 -1.49129085  1.86232967]
 [-0.50012642 -0.24755435  0.42571227  0.19207563]] 

bias :
[[ 0.70517626]
 [ 1.51605412]
 [ 1.03011131]
 [-0.21782916]] 

[[ 1.30923064]
 [-1.5072601 ]] 



### İleri Besleme

![network.jpg](network.jpg)


![noronislem.JPG](noronislem.JPG)


Yukarıdaki yapay sinir ağı, 3 katmandan oluşmaktadır.

> Birinci katman, girdi olarak $x_1$, $x_2$, $x_3$ verisini alir. 

ilk katmana özel olarak $a_i(1) = z_i(1) = x_i$dir. 

> Daha sonraki katmanlardaki nöronlar, bir önceki katmanın ağırlıklı toplamını girdi olarak kabul eder.



Genel olarak,

> $z_j(t) = \sum_i w_{j,i}(t) \times a_i(t-1) + b_j(t)$

mesela  $z_4(2) = w_{4,1}(2) \times a_1(1) + w_{4,2}(2) \times a_2(1) + w_{4,3}(2) \times a_3(1) + b_4(2)$dir.

> $w_{j,i}(t)$: $\hspace{1cm}$ $(t-1)$'inci katmandaki $i$'nci nörondan $(t)$'inci katmandaki $j$nci nörona olan bağlantının ağırlık değeridir.

Aynı şekilde

> $b_{j}(t)$: $\hspace{1cm}$ $(t)$'inci katmandaki $j$'nci nörona ait bias değeridir

Bir nöronun çıktısı, sigmoid fonksiyonuna net girdi değeri verilerek hesaplanır.

> $a_j(t) = \sigma(z_j(t)) = \frac{1}{1 + e^{z_j(t)}}$


#### İleri Besleme Algoritması

Ağımız verilen girdi x ve ağırlık w, bias b değerlerine göre bir çıktı üretir. Vektörize edilerek yapılan işlem

> $z(t) = w(t) \cdot a(t-1) + b(t)$

> $a(t) = \sigma(z(t))$

Her bir katmandaki nöronlar, önceki katmanlardaki nöron çıktılarını ağırlıklarıyla çarpıp son olarak bias(çapa ya da referans) değeri ekleyerek net girdi olan $z$ değerini bulurlar. Sonraki işlem ise, sigmoid ile çıktı değerini hesaplamaktır.


![vektorize.jpg](vektorize.jpg)

In [11]:
def ileribesleme(a, agirlik, bias):
    """Katman katman yeni a degerleri hesaplaniyor"""
    for w, b in zip(agirlik, bias):
        z = np.dot(w, a)+b
        a = sigmoid(z)
    return a

In [12]:
katmanlar = [2, 3, 1]
agirlik, bias = ag(katmanlar)

girdi = [[0], 
         [0]]
print(ileribesleme(girdi, agirlik, bias))

[[ 0.39334822]]


### Yardımcı Türevler

İleri Besleme algortimasına bakarak şu türevleri bulalım, bunlar bize geri besleme algoritmasında yardımcı olacak

> 
$$
\frac{d z_j(t+1)}{d z_i(t)} = 
\frac{ d (\sum_i w_{j,i}(t+1) \times \sigma(z_i(t)) + b_j(t+1))}{d z_i(t)}
=
w_{j,i}(t+1) \times \sigma'(z_i(t))
$$

![back1.jpg](back1.jpg)

### Geri Besleme

Geri besleme algoritmasındaki kilit nokta,
bir nöronun ait girdideki değişim, hatayı nasıl etkiler sorusudur.


$$
\Delta_j(t) =
\frac{d Hata}{d z_j(t)}
$$

Bu soruya vereceğimiz cevap ile, ağırlık ve bias değerlerini hatayı minimize edecek şekilde nasıl güncelleyeceğimizi bulacağız. Ve tabi gene gradyan iniş yöntemini kullanacağız.

$$Hata = \frac{1}{2} \sum_j (y_j - a_j(T))^2$$

$T$ ağdaki çıktı katmanıdır. Bu durumda çıktı katmanındaki j'inci nöronun hataya etkisi

$$
\Delta_j(T) =
\frac{d Hata}{d z_j(T)} = \frac{d Hata}{d a_j(T)} \frac{d a_j(T)}{d z_j(T)}
= (a_j(T)-y_j ) \frac{d a_j(T)}{d z_j(T)}
= (a_j(T)-y_j ) \sigma'(z_j(T))
$$ 

Çıktı katmanındaki tüm nöronların hataya olan etkisini vektörize edersek

> 
$$
\Delta(T) = \frac{d Hata}{d z(T)} = (a(T)-y) \sigma'(z(T))
$$

Dikkat ederseniz, vektörize ettiğimizde indislerden kurtuluyoruz. Peki bir ara katmandan, önceki ara katmana hata nasıl yayılır?

> 
$$
\Delta_i(t) =
\frac{d Hata}{d z_i(t)} 
= 
\sum_j \frac{d Hata}{d z_j(t+1)} \frac{d z_j(t+1)}{d z_i(t)}
= 
\sum_j \Delta_j(t+1) \frac{d z_j(t+1)}{d z_i(t)}
$$


Unutmayın ki, $(t)$'inci katmandaki nöron $i$'ye ait bir hata
$(t+1)$'inci katmandaki tüm $j$ nöronlarını etkiler. Sonuç olarak,

> 
$$
\Delta_i(t) =
\sum_j \Delta_j(t+1) w_{j,i}(t+1) \times \sigma'(z_i(t))
$$

$(t+1)$'inci katmandan $(t)$'inci katmana hatanın akışını vektörize edersek,

$$
\Delta(t) =  w^T(t+1) \cdot  \Delta(t+1) \times \sigma'(z(t))
$$

![back2.jpg](back2.jpg)

### Nöronlardaki hatanın güncellenmesi

Çıktı katmanında
$$
\Delta(T) = \frac{d Hata}{d z(T)} = (a(T)-y) \sigma'(z(T))
$$

$(t+1)$'inci ara katmandan $(t)$'inci ara katmana hatanın akışı,

$$
\Delta(t) =  w^T(t+1) \cdot  \Delta(t+1) \times \sigma'(z(t))
$$

### Ağırlık ve bias değerlerinin güncellenmesi

Hatayı minimize eden en iyi parametreleri arıyoruz.

Ağırlıktaki değişim
$$
\frac{d Hata}{d w_{ji}(t)} = \frac{d Hata}{d z_{j}(t)} \frac{d z_{j}(t)}{d w_{ji}(t)} 
= \Delta_{j}(t)  a_i(t-1)
$$

Biasdeki değişim
$$
\frac{d Hata}{d b_{j}(t)} = \frac{d Hata}{d z_{j}(t)} \frac{d z_{j}(t)}{d b_{j}(t)} 
= \Delta_{j}(t) 
$$

Geri Besleme algoritması neden hızlıdır?

> Dikkat ederseniz $\Delta_{j}(t)$ değerini sadece bir kez hesaplayıp, yeniden hesaplamadan bir çok yerde (Ağırlıktaki ve Biasdeki değişimde) tekrar tekrar kullanıyoruz. Bu bize hız kazandırıyor.

![back3.jpg](back3.jpg)

In [13]:
def geribesleme(x,y, agirlik, bias): #girdi, cikti
    delta_b = [np.zeros(b.shape) for b in bias]
    delta_w = [np.zeros(w.shape) for w in agirlik]
    
    a = x
    A = [a] # a degerleri
    Z = []  # z degerleri
    for w, b in zip(agirlik, bias):# z ve a degerlerini depolayalim
        z = np.dot(w, a) + b
        a = sigmoid(z)
        Z.append(z)
        A.append(a)
    
    hata = A[-1] - y # en son katmandaki hata
    delta = hata * sigmoid_turevi(Z[-1])
    delta_b[-1] = delta
    delta_w[-1] = np.dot(delta, A[-2].T)
    for k in range(2, len(katmanlar)):
        delta = np.dot(agirlik[-k+1].T, delta) * sigmoid_turevi(Z[-k])
        delta_b[-k] = delta
        delta_w[-k] = np.dot(delta, A[-k-1].T)
    return (delta_b, delta_w)  

In [14]:
def gradyan_inis(orneklem, adim, agirlik, bias):
    delta_b = [np.zeros(b.shape) for b in bias]
    delta_w = [np.zeros(w.shape) for w in agirlik]
    
    for x, y in orneklem:
        # geri besleme tek bir x-y cifti icin w-b parametrelerinde degisimi verir
        delta_bxy, delta_wxy = geribesleme(x, y, agirlik, bias)
        # orneklemdeki farkli x-y ciftleri icin w-b degisimleri toplanir
        delta_b = [nb+dnb for nb, dnb in zip(delta_b, delta_bxy)]
        delta_w = [nw+dnw for nw, dnw in zip(delta_w, delta_wxy)]
    
    # w-b parametreleri turevlerinin tersi yonde guncelleniyor (gradyan_inis)    
    agirlik = [w-(adim/len(orneklem))*nw for w, nw in zip(agirlik, delta_w)]
    bias = [b-(adim/len(orneklem))*nb for b, nb in zip(bias, delta_b)]
    return agirlik, bias

In [15]:
def ogrenme(veri, agirlik, bias, epochs = 30, sayi = 10, adim = 0.5, test_data=None):
    """ epochs: kac kez tum veri kullaniacak
        icinde sayi kadar ornek bulunan orneklemler(mini-batch)
        gradyan-iniste kullaniliyor. Agirlik ve bias degerleri guncelleniyor.
        Ogrenme:
            en iyi Agirlik ve bias degerelerini bulmak
    """
    veri = list(veri)
    n = len(veri)

    if test_data:
        test_data = list(test_data)
        n_test = len(test_data)

    for j in range(epochs):
        random.shuffle(veri)
        for orneklem in [veri[k:k+sayi] for k in range(0, n, sayi)]:
            agirlik, bias = gradyan_inis(orneklem, adim, agirlik, bias)
        if test_data:
            print("Epoch {} : {} / {}".format(j, tahmin(test_data, agirlik, bias), n_test));
        else:
            print("Epoch {} complete".format(j))
    return agirlik, bias

In [16]:
def tahmin(test_data, agirlik, bias):
    """Return the number of test inputs for which the neural
    network outputs the correct result. Note that the neural
    network's output is assumed to be the index of whichever
    neuron in the final layer has the highest activation."""
    test_results = [(np.argmax(ileribesleme(x, agirlik, bias)), y) for (x, y) in test_data]
    return sum(int(x == y) for (x, y) in test_results)

## Calisma


In [17]:
# 1 ile 10 arasinda 1000 adet x1 ve x2
X = np.random.randint(10, size=(10000,2))
print(X[1:6,])

[[8 1]
 [9 9]
 [7 3]
 [9 1]
 [8 6]]


In [18]:
# ilk kolon ikinci kolondan buyukse, y=1
y = (X[:,0] > X[:,1]) * 1
print(y[1:6,])

[1 0 1 1 1]


In [19]:
veri = zip(X,y)
print(veri)

In [21]:
katmanlar = [2, 6, 1]
agirlik, bias = ag(katmanlar)
agirlik, bias = ogrenme(veri, agirlik, bias, epochs = 10, sayi = 10, adim = 0.05)

Epoch 0 complete
Epoch 1 complete
Epoch 2 complete
Epoch 3 complete
Epoch 4 complete
Epoch 5 complete
Epoch 6 complete
Epoch 7 complete
Epoch 8 complete
Epoch 9 complete


In [22]:
agirlik, bias

([array([[-0.22594033,  0.41448543],
         [-1.79304226,  0.73889988],
         [-0.40693725, -1.79231886],
         [-0.05900289, -1.88539912],
         [ 0.92139126, -1.12725437],
         [-0.34777916,  1.68350194]]),
  array([[ 1.14515488,  2.00346069,  0.10789699,  1.30205599, -0.80644706,
          -1.53902317]])],
 [array([[-0.95496157],
         [ 1.47850848],
         [-0.09078403],
         [ 0.53993107],
         [-1.27719393],
         [-0.70266968]]), array([[-0.87388767]])])

In [25]:
girdi = np.array([100,10])
print(ileribesleme(girdi, agirlik, bias))

[[ 0.29444602  0.294446    0.294446    0.294446    0.7923601   0.29444604]]


In [26]:
girdi = np.array([0,100])
print(ileribesleme(girdi, agirlik, bias))

[[ 0.7923601  0.7923601  0.294446   0.294446   0.294446   0.7923601]]


In [27]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
training_data = list(training_data)

In [28]:
t = training_data[1:2]
np.shape(training_data[1:2])
#print(training_data)


(1, 2)

In [29]:
katmanlar = [784, 30, 10]
agirlik, bias = ag(katmanlar)

In [30]:
ogrenme(training_data, agirlik, bias, 3, 10, 3.0, test_data=test_data)

Epoch 0 : 9099 / 10000
Epoch 1 : 9252 / 10000
Epoch 2 : 9279 / 10000


([array([[ 0.52692743, -0.51635572, -0.46859854, ...,  0.74222485,
          -0.95390091,  2.1680461 ],
         [-0.68995664, -1.37008425,  0.77278083, ...,  0.90109157,
           0.51903637,  1.38607988],
         [ 0.31347105, -0.35427878,  0.51644072, ...,  1.0237791 ,
           0.04055459,  1.62078105],
         ..., 
         [-0.069219  ,  0.62464401,  1.29832614, ...,  0.22958059,
           2.15770912,  0.1137555 ],
         [ 1.24345394, -0.7471736 , -0.04299237, ..., -0.56584895,
           1.16007671, -1.35362772],
         [ 1.48145274, -0.07407779,  0.80288164, ...,  0.16352206,
           1.19552868, -0.46035591]]),
  array([[ -1.39461550e+00,  -1.28596609e+00,  -2.48632042e+00,
            1.37009517e+00,  -3.06951370e+00,   1.00541388e+00,
           -3.96591398e-03,  -2.52547697e+00,   9.86664953e-01,
            1.36722163e+00,   2.31152624e+00,   6.01503856e-01,
           -1.81068475e+00,  -2.22797086e+00,   4.29981998e+00,
            6.01764616e-01,  -1.5373411

In [31]:
agirlik, bias = gradyan_inis(training_data[1:10],adim = 0.5, agirlik = agirlik, bias = bias)

In [32]:
agirlik, bias

([array([[ 0.52692743, -0.51635572, -0.46859854, ...,  0.74222485,
          -0.95390091,  2.1680461 ],
         [-0.68995664, -1.37008425,  0.77278083, ...,  0.90109157,
           0.51903637,  1.38607988],
         [ 0.31347105, -0.35427878,  0.51644072, ...,  1.0237791 ,
           0.04055459,  1.62078105],
         ..., 
         [-0.069219  ,  0.62464401,  1.29832614, ...,  0.22958059,
           2.15770912,  0.1137555 ],
         [ 1.24345394, -0.7471736 , -0.04299237, ..., -0.56584895,
           1.16007671, -1.35362772],
         [ 1.48145274, -0.07407779,  0.80288164, ...,  0.16352206,
           1.19552868, -0.46035591]]),
  array([[ -1.58517494e+00,   9.71274819e-01,  -8.60991307e-01,
            1.44966624e+00,  -7.02485450e-01,   1.26727611e+00,
           -7.90720736e-01,  -6.50369651e-01,  -2.95876518e-01,
            6.47548039e-01,   3.73965651e-01,   1.52615766e-02,
            8.98367183e-01,   6.44631181e-01,   8.32465851e-01,
            4.47205372e-01,  -4.3398752